# Similaridade entre Produtos
### Instale dependências necessarias via:
```
pip instal requirements.txt
```

In [1]:
import logging
import warnings
import numpy as np
import pandas as pd

from utils import data_manager, fasttext_model

warnings.filterwarnings("ignore",category=DeprecationWarning)

### Instalando lista de _stopwords_ via nltk.
O uso do instalador nltk é conveniente para remoção de _stopwords_ uma vez que o mesmo gera um arquivo local, nos possibilitando adicionar/remover palavvras da lista vigente.

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/robsonortz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('portuguese')

### Leitura e tratamento dos dados

Aplicamos o processo de normalização de _tokens_ (aplicamos _lower case_ nas sentenças, removemos caracteres especiais e _tokens_ numéricos) e remoção de _stopwords_.

Optamos por considerar apenas títulos que contenham, ao final do tratamento, ao menos 2 _tokens_. Pois o conjunto textual é composto por textos curtos e para evitar problemas com o modelos de _embedding_ selecionado (_fasttext_), que geralmente se desempenha melhor em textos longos, obtamos por remover títulos que resultam em uma única palavra. 

Modelos pré-treinados, utilizados para _fine tuning_ de processos de NLP, são mais adequados para tais dados. No entanto, boa parte desses, que realizam seu pré-treino em _corpus_ em português ou de multilinguagem, são caros e geralemente utlizam recursos de GPU.

In [4]:
dm = data_manager.DataManager()

train_items_titles = dm.text_tokenizer(list(pd.read_csv('data/items_titles.csv').values.T[0]), stopwords)
test_items_titles = dm.text_tokenizer(list(pd.read_csv('data/items_titles_test.csv').values.T[0]), stopwords)
len(train_items_titles), len(test_items_titles)

(29940, 9979)

### Treinamento do modelo _fasttext_



In [5]:
config = dm.set_configurations('config')
ftm = fasttext_model.ModelManager(config)

In [8]:
ftm.train_model(train_items_titles)

### Obtenção dos embedding para os dados de teste

In [7]:
embedding_list = ftm.vector_matrix_embedding(test_items_titles)

O processo de geração de _embedding_ pode demorar, se necessário usar o código de _checkpoint_ abaixo para evitar perda de informação dos _embedding_. 

Após salvar os dados, podemos ler, a qualquer momento, o `.csv` completo. Lembrondo que devemos tratá-lo para que atenda a estrutura correta de colunas.

In [28]:
##checkpoint

# data_test_embedding = pd.DataFrame({
#     'product_name':list(map(' '.join, test_items_titles))
# })

# data_test_embedding = pd.concat([data_test_embedding, pd.DataFrame(embedding_list)], axis=1)
# data_test_embedding.to_csv('data/data_test_embedding_checkpoint.csv')

In [31]:
data_test_embedding = pd.DataFrame({
    'product_name':list(map(' '.join, test_items_titles)),
    'embeddings_vector':list(embedding_list)
})

data_test_embedding

,product_name,embeddings_vector
0,tenis olympikus esporte valente masculino kids,"[1.6316595673561096, -2.2509637754410505, 0.59..."
1,bicicleta barra forte samy marchas cubo rolamento,"[0.987981466576457, 0.9649230418726802, 0.7371..."
2,tenis usthemp slip on tema tico labrador,"[-4.3989940993487835, -0.7319259578362107, -3...."
3,tenis casual feminino moleca tecido tie dye,"[4.7329486683011055, -0.6628666631877422, -0.6..."
4,tenis star baby sapatinho conforto brinde,"[-1.232651636004448, -1.4965157099068165, -0.0..."
...,...,...
9974,chuteira futsal oxn velox infantil,"[1.0121160000562668, -0.6645795181393623, -0.0..."
9975,sapatenis casual masculino estiloso horas conf...,"[-2.4321695286780596, -3.1090461295098066, -2...."
9976,tenis feminino infantil molekinha tie dye,"[5.016561217606068, -0.2409418597817421, 0.598..."
9977,tenis feminino leve barato ganhe colchonete tr...,"[1.1084097921848297, -3.5079212710261345, 0.59..."


### Resultados

Enquanto named_rows apresenta os três produtos mais similares ao produto da chave do dicionário, o _output_ simil apresenta os escores de similaridade via KDTree.

In [54]:
named_rows, simil = dm.kdtree_similarity(data_test_embedding)

Pandas Apply:   0%|          | 0/9979 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/9979 [00:00<?, ?it/s]

In [63]:
final_list = []

for result in simil:
    final_list.append(
        [data_test_embedding['product_name'][result[0]],data_test_embedding['product_name'][result[1]],result[2]]
    )

results = pd.DataFrame(final_list, columns=['ITE_ITEM_TITLE', 'ITE_ITEM_TITLE', 'score(0,1)'])
results.to_csv('similarity_results')

In [64]:
results

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,"score(0,1)"
0,tenis usthemp one art maltes carolina spina,tenis usthemp one tema tico vaquinha,0.717470
1,tenis usthemp one art maltes carolina spina,tenis usthemp one tema tico design,0.734706
2,tenis usthemp one art maltes carolina spina,tenis usthemp one tema tico girafa,0.729569
3,tenis usthemp one art maltes carolina spina,tenis usthemp one tema tico gato himalaio,0.739258
4,tenis usthemp one art maltes carolina spina,tenis usthemp one vegano casual estampa libra,0.752781
...,...,...,...
2572946,tenis juvenil lol,tenis infantil lol lancamento envio imediato b...,0.760908
2572947,tenis fila formation feminino,tenis fila squall feminino,0.601745
2572948,tenis fila formation feminino,tenis feminino fila extase original,0.518904
2572949,tenis fila formation feminino,tenis fila feminino formation branco revendedo...,0.367734
